# Getting started with AI Agent Service in Python

## Prep development environment

First you need to create new Python virtual environment above on right

In [ ]:
pip install "semantic-kernel[azure]"

Next create .env file with below configuration 👇
```
AZURE_AI_AGENT_PROJECT_CONNECTION_STRING = "<example-connection-string>"
AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME = "<example-model-deployment-name>"
```

And add required imports.

In [5]:
import asyncio

from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

## Let's make some cooking 🧑🏽‍🍳 🔥

In [8]:
# Simulate a conversation with the agent
USER_INPUTS = [
    "Why is the sky blue?",
]

async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(creds) as client
):
    # 1. Retrieve the agent definition based on the `agent_id`
    # Replace the "your-agent-id" with the actual agent ID
    # you want to use.
    agent_definition = await client.agents.get_agent(
        agent_id="asst_oHmRHEfCufdamKksLlF9H95k",
    )

    # 2. Create a Semantic Kernel agent for the Azure AI agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
    )

    # 3. Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread = None

    try:
        for user_input in USER_INPUTS:
            print(f"# User: '{user_input}'")
            # 4. Invoke the agent for the specified thread for response
            response = await agent.get_response(messages=user_input, thread=thread)
            print(f"# {response.name}: {response}")
    finally:
        # 5. Cleanup: Delete the thread and agent
        await thread.delete() if thread else None
        # Do not clean up the agent so it can be used again

# User: 'Why is the sky blue?'
# Agent734: The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight passes through the Earth's atmosphere, it interacts with air molecules and small particles. Sunlight, or white light, is composed of different colors, each with varying wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.

As sunlight encounters the atmosphere, the shorter wavelengths (blue and violet) are scattered in all directions by the gases and particles. However, our eyes are more sensitive to blue light than to violet light, and some of the violet light is absorbed by the ozone layer, which means we perceive the sky as blue.

When the sun is lower in the sky, such as during sunrise or sunset, the light has to pass through a greater thickness of the atmosphere. This increased distance scatters the shorter wavelengths more and allows the longer wavelengths (reds and oranges) to dominate, resulting in the b

## Let's make more fun with multi-agent 🥳

In [ ]:
import asyncio

from azure.identity.aio import DefaultAzureCredential
from semantic_kernel import Kernel
from semantic_kernel.agents import AzureAIAgent, AgentGroupChat
from semantic_kernel.contents import AuthorRole, ChatHistoryTruncationReducer
from semantic_kernel.functions import KernelFunctionFromPrompt
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
    TerminationStrategy
)

### Let's use some glue 👀, create Selection Strategy and Termination Strategy

Let's set some job description

In [34]:
JOB_DESCRIPTION = """ITMAGINATION helps its Clients by becoming a true extension of their software and data development capabilities. Through our readily set up, comprehensive, and self-governing teams, we let our Clients focus on their business while we make sure that their software products and data tools scale up accordingly and with outstanding quality.

We are looking for experienced team players to fill the Operations Engineer - Support Line 2 position and participate in our up-and-coming project from the chemical manufacturing industry.

You can expect:
Working with a highly skilled team of professionals
Monitoring & supporting production systems


Requirements
Min. 5 years of experience working in DevOps or SRE
Experience with CI/CD tools (e.g., Azure DevOps)
Min. 2 years working experience with Azure
Knowledge on VM’s, Appservices, App Gateways, Keyvaults, Functions,VNETs, Logicapps, Log analytic queries KQL, App insights, etc.
Experience in Azure Native monitoring tools like – Az Monitor, Appinsights, Loganalytics, Dashboards, Grafana, Prmetheus, Solarwinds, Zabbix, etc.
Ability to write SQL/Postgres queries
Understanding of all aspects of an application stack and associated technologies (Network, OS, Web, App, DB, Storage)
GDPR and PHI/PII regulations awareness
Good understanding/experience in Incident Management – Change Management and Problem Management
Excellent English skills


Benefits
Fully remote work model
Professional training programs – including Udemy and other development plans
Work with a team that’s recognized for its excellence. We’ve been featured in the Deloitte Technology Fast 50 & FT 1000 rankings. We’ve also received the Great Place To Work® certification for five years in a row"""

In [41]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()

def create_kernel() -> Kernel:
    """Creates a Kernel instance with an Azure OpenAI ChatCompletion service."""
    kernel = Kernel()
    return kernel

async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(creds) as client
):
    # Create a single kernel instance for all agents.
    kernel = create_kernel()

    managerDefinition = await client.agents.get_agent("asst_H46WKAqAJtIGvr5S1hGqKqfA")
    reviewerDefinition = await client.agents.get_agent("asst_oHmRHEfCufdamKksLlF9H95k")
    skillsDefinition = await client.agents.get_agent("asst_GPBPFdJxR7Vi3VdP7vZnKqch")

    manager_agent = AzureAIAgent(
        client=client,
        definition=managerDefinition,
    )

    job_reviewer = AzureAIAgent(
        client=client,
        definition=reviewerDefinition,
    )

    skills_agent = AzureAIAgent(
        client=client,
        definition=skillsDefinition,
    )

    # selection_function = KernelFunctionFromPrompt(
    #     function_name="selection",
    #     prompt=f"""
    #         Examine the provided RESPONSE and choose the next participant.
    #         State only the name of the chosen participant without explanation.
    #         Never choose the participant named in the RESPONSE.

    #         Choose only from these participants:
    #         - {manager_agent}
    #         - {skills_agent}
    #         - {job_reviewer}

    #         Rules:
    #         - If RESPONSE is user input, it is {job_reviewer}'s turn.
    #         - If RESPONSE is by {job_reviewer}, it is {skills_agent}'s turn.
    #         - If RESPONSE is by {skills_agent}, it is {manager_agent}'s turn.

    #         RESPONSE:
    #         {{{{$lastmessage}}}}
    #         """,
    # )

    # Define a termination function where the reviewer signals completion with "yes".
    # termination_keyword = "yes"

    # termination_function = KernelFunctionFromPrompt(
    #     function_name="termination",
    #     prompt=f"""
    #         Examine the RESPONSE and determine whether the content has been deemed satisfactory.
    #         If the content is satisfactory, respond with a single word without explanation: {termination_keyword}.
    #         If specific suggestions are being provided, it is not satisfactory.
    #         If no correction is suggested, it is satisfactory.

    #         RESPONSE:
    #         {{{{$lastmessage}}}}
    #         """,
    # )

    history_reducer = ChatHistoryTruncationReducer(target_count=5)

    chat = AgentGroupChat(
        agents=[manager_agent, job_reviewer, skills_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[manager_agent], maximum_iterations=10),
    )

    try:
        await chat.add_chat_message(JOB_DESCRIPTION)
        print(f"# {AuthorRole.USER}: '{JOB_DESCRIPTION}'")
        # Invoke the chat
        async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")
    finally:
        # Cleanup
        await chat.reset()

# AuthorRole.USER: 'ITMAGINATION helps its Clients by becoming a true extension of their software and data development capabilities. Through our readily set up, comprehensive, and self-governing teams, we let our Clients focus on their business while we make sure that their software products and data tools scale up accordingly and with outstanding quality.

We are looking for experienced team players to fill the Operations Engineer - Support Line 2 position and participate in our up-and-coming project from the chemical manufacturing industry.

You can expect:
Working with a highly skilled team of professionals
Monitoring & supporting production systems


Requirements
Min. 5 years of experience working in DevOps or SRE
Experience with CI/CD tools (e.g., Azure DevOps)
Min. 2 years working experience with Azure
Knowledge on VM’s, Appservices, App Gateways, Keyvaults, Functions,VNETs, Logicapps, Log analytic queries KQL, App insights, etc.
Experience in Azure Native monitoring tools like –